### Dealing with Missing Data
* Team: Jonathan Tan, Lance Dacy, Reannan Mcdaniel, Shawn Jung
* Last Update: 7/6/2020
* Based on California Housing Price data from Scikit Learn, we would like to explore ways to handle missing data

In [31]:
# Import package dependencies
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
#from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing

In [32]:
# load dataset
cali = fetch_california_housing()
print(cali.data.shape)


(20640, 8)


In [33]:
print(cali.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR'])


In [34]:
# View the data descriptions
print(cali.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [35]:
cali.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [36]:
# Convert the matrix to pandas dataframe
X = pd.DataFrame(cali.data)
X.columns = cali.feature_names
y = cali.target
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [37]:
X.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


## Start by fitting a Linear Regression model to the full dataset

**Create a training and testing split (ex., 70/30-split)**

In [38]:
# Create training and testing sets (cross-validation not needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)


In [39]:
# Fit a linear regression to the training data
reg = LinearRegression(normalize=True).fit(X_train, y_train)
print(reg.score(X_train, y_train))
print(reg.coef_)
print(reg.intercept_)
print(reg.get_params())

0.609370412027382
[ 4.44870466e-01  9.55004561e-03 -1.21991503e-01  7.79144696e-01
 -7.68990809e-08 -3.29948505e-03 -4.19131153e-01 -4.34103468e-01]
-37.0820109390799
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}


In [40]:
# Find the variable with the largest "normalized" coefficient value
print('The abs(max) coef-value is {}'.format(max(reg.coef_))) # Positive Max
#print('The abs(max) coef-value is {}'.format(max(reg.coef_, key=abs))) # ABS Max
max_var = max(reg.coef_) # Positive Max
#max_var = max(reg.coef_, key=abs) # ABS Max
var_index = reg.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(cali.feature_names[var_index]))

The abs(max) coef-value is 0.7791446958109836
The variable associated with this coef-value is AveBedrms


# Step 1
Question 1. What is the loss and what are the goodness of fit parameters?  This will be our baseline for comparison.

In [41]:
y_pred = reg.predict(X_test)

orig_mae = mean_absolute_error(y_test,y_pred)
orig_mse = mean_squared_error(y_test,y_pred)
orig_rmse_val = np.sqrt(orig_mse)
orig_r2 = r2_score(y_test,y_pred)
print("MAE: %.3f"%orig_mae)
print("MSE:  %.3f"%orig_mse)
print("RMSE:  %.3f"%orig_rmse_val)
print("R2:  %.3f"%orig_r2)

MAE: 0.530
MSE:  0.537
RMSE:  0.733
R2:  0.597


We will define the prediction result placeholder, resd_frame, and add the first model to the dataframe

In [42]:
res_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'rmse':orig_rmse_val, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'rmse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.529571,0.536969,0.732781,0.597049,NaN,NaN,NaN,NaN


# Step 2

**Here we can randomly sample the full dataset and replace a single column's values** <BR>
At first, let's define a function that returns mae, mse, rmse and R2 

In [43]:
def regression_func(xdata, ydata, data_desc='none', impute_method='none'):
    ''' This function run linear regression with input data, and returns a dataframe of goodness of fit metrics '''

    X_train, X_test, y_train, y_test = train_test_split(xdata, ydata, test_size=0.33, random_state=42, shuffle=True)
    reg = LinearRegression(normalize=True).fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    mae = mean_absolute_error(y_test,y_pred)
    mse = mean_squared_error(y_test,y_pred)
    rmse_val = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    temp_frame = pd.DataFrame({'data':data_desc,
                   'imputation':impute_method,
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae - orig_mae,
                   'mse_diff':mse - orig_mse,
                   'rmse_diff':rmse_val - orig_rmse_val,
                   'R2_diff':r2 - orig_r2
                   }, index=[0])

    return temp_frame

It works ok when tested with the original data

In [44]:
regression_func(X, y)

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,none,none,0.529571,0.536969,0.732781,0.597049,0.0,0.0,0.0,0.0


Now define a function to nullify a column, and infer the missing value from the median of other non-missing values

In [45]:
def impute_by_median(xdata, col_name, null_frac):
    ''' this function nullify the fraction of the input datafarme's column, and impute it by the median value of remaining non-null values. The result is returned as a data frame'''
    
    in_sample = xdata.sample(frac=null_frac, random_state=42)
    out_sample = xdata[~xdata.isin(in_sample)].dropna()
    in_sample[col_name] = np.nan
    in_sample[col_name] = in_sample[col_name].fillna(out_sample[col_name].median())
    imputed_data = pd.concat([in_sample, out_sample]).sort_index()

    return imputed_data




Question 2: In each case 1%, 5%, 10%, 20%, 33%, 50% perform a fit with the imputed data and compare the loss and goodness of fit to your baseline.  Note: you should have (6) models to compare against your baseline at this point.

We will try 'impute mssing value by median' method for 'AveRooms' variable

In [46]:
fractions = [0.01, 0.05, 0.1, 0.2, 0.33, 0.5]

for fraction in fractions:
    X_imputed = impute_by_median(X, 'AveRooms', fraction)
    data_desc='Nullify ' + str(fraction*100) + '%'
    temp_result = regression_func(X_imputed, y, data_desc , impute_method='median') 

    #print(temp_result)
    res_frame = pd.concat([res_frame, temp_result])

In [47]:
res_frame = res_frame.reset_index(drop=True)
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,0.529571,0.536969,0.732781,0.597049,NaN,NaN,NaN,NaN
1,Nullify 1.0%,median,0.530093,0.538481,0.733813,0.595914,0.000522,0.001512,0.001031,-0.001135
2,Nullify 5.0%,median,0.531264,0.540778,0.735376,0.594191,0.001693,0.003809,0.002594,-0.002858
3,Nullify 10.0%,median,0.534101,0.556453,0.745958,0.582428,0.004530,0.019485,0.013176,-0.014622
4,Nullify 20.0%,median,0.540786,0.649251,0.805761,0.512791,0.011215,0.112282,0.072979,-0.084258
5,Nullify 33.0%,median,0.547262,0.673146,0.820455,0.494860,0.017691,0.136177,0.087673,-0.102190
6,Nullify 50.0%,median,0.537447,0.562518,0.750012,0.577877,0.007876,0.025549,0.017230,-0.019172
